In [1]:
import os
import numpy as np
import pandas as pd
from typing import Optional, List, Tuple, Callable
from ast import literal_eval

In [2]:
d = pd.read_excel("../data/Masterlist.xlsx", sheet_name="Variables")
d.head()

,Name,Class,Code,Unit,Attribute,Sector,Commodity,Process,Category1,Category2,Category3,Category4,Com_Sector,Com_Type1,Com_Type2,Com_Type3
0,Emissions|CO2,single_var,O-1-1,kt/yr,VAR_FOut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Environment,CO2,NaN
1,Emissions|Industry|CO2,single_var,O-1-2,kt/yr,VAR_FOut,NaN,"INDCO2N, INDCO2P",NaN,NaN,NaN,NaN,NaN,NaN,Environment,CO2,NaN
2,Emissions|Transport|CO2,single_var,O-1-3,kt/yr,VAR_FOut,NaN,"TRACO2N, TRACO2P",NaN,NaN,NaN,NaN,NaN,NaN,Environment,CO2,NaN
3,Emissions|Residential|CO2,single_var,O-1-4,kt/yr,VAR_FOut,NaN,"RSDCO2N, RSDCO2P",NaN,NaN,NaN,NaN,NaN,NaN,Environment,CO2,NaN
4,Emissions|Commercial|CO2,single_var,O-1-5,kt/yr,VAR_FOut,NaN,"COMCO2N, COMCO2P",NaN,NaN,NaN,NaN,NaN,NaN,Environment,CO2,NaN


In [3]:
import os, sys
sys.path.append("..")

from src.utils import vd_reader
from src.framework.powerframe import PowerFrame
from src.framework.varclass import SingleVar
from src.utils.filters import filter_any, filter_from_csv


inp_file = "../data/8GT_Bal.VD"
masterlist = "../data/Masterlist.xlsx"

%time data = vd_reader(path = inp_file, masterlist_path=masterlist)

Found masterlist at ../data/Masterlist.xlsx
CPU times: user 22.9 s, sys: 3.27 s, total: 26.2 s
Wall time: 28.3 s


In [4]:
# A class is created, which contains both the data, the summarized data, the name and other information
# Each variable is then a characteristic of this class
file_name = "../data/8GT_Bal.VD"
scenario_name = os.path.split(file_name)[-1].split(".")[0]
region_list = ["DER", "BW"]

In [5]:
class single_var(object):
    # Create a list in which all attributes of the class are stored, so a loop can be run over it later on
    _registry = []
    # The init function is always called. Here the assignments of the variables are defined within the class.
    # The variables must be given always in the order as in the next line, "self" can be ignored thereby.
    def __init__(self, template, var_name, unit, df_rawdata):
        self._registry.append(self)
        self.var_name = var_name
        # df rawdata wird mit oben stehender Filter Methode erstellt und beeinhaltet alle einzelnen Zeilen aus der VD Datei
        self.df_rawdata = df_rawdata
        # Wenn template "True" ist, dann wird die Variable im Template ausgegeben, sonst auf "False" setzen
        self.template = template

        self.unit = unit
        self.yearlist = []
        for year in df_rawdata['Period'].unique():
            try:
                year = int(year)
            except:
                continue
            self.yearlist.append(str(year))
        
        # year_dict with all years initialized with 0 value
        self.year_dict = {str(year):'0' for year in self.yearlist}
        dict_df = {"Model" : "TIMES",
                   "Scenario": scenario_name,
                   "Region": "Placeholder",
                   "Variable": self.var_name,
                   "Unit": self.unit}
        dict_df.update(self.year_dict)
        df_temp = pd.DataFrame(data=dict_df, index = [0])

        df_region_sum = []
        # Creates for each region and year the sum of all values and thus the total value of the variable
        for region in region_list:
            dict_df = {
                "Model" : "TIMES PanEU v1.0",
                "Scenario" : scenario_name,
                "Region" : region,
                "Variable": self.var_name, 
                "Unit": self.unit}
            dict_df.update(self.year_dict)

            df_temp_region = pd.DataFrame(data=dict_df, index = [0])
            mask_region = df_rawdata.Region == region
            df_region = df_rawdata.loc[mask_region]

            for year in self.yearlist:
                mask_year = df_region.Period == year
                df_year = df_region.loc[mask_year]
                year_sum = df_year["PV"].sum()
                df_temp_region[year] = year_sum
                
            df_region_sum.append(df_temp_region)

        # Die oben erstellte und befüllte Tabelle wird immer als Variable.results gespeichert 
        self.results = pd.concat(df_region_sum)


    def add_Germany(self):
        # Diese Funktion ist notwendig, um die beiden deutschen Regionen DER und BW zu addieren. Dabei bleiben die Regionen vorerst enthalten
            # Einfache Möglichkeit, um 2 versch. Filter miteinander zu kombinieren
        mask1 = self.results.Region == "DER"
        mask2 = self.results.Region == "BW"
        df_temp = self.results.loc[mask1 | mask2]


        #df_temp = self.results.loc[self.results["Region"] == ["DER", "BW"]]
        dict_df = {
            "Model": "TIMES PanEU v1.0",
            "Scenario": scenario_name,
            "Region": "DEU",
            "Variable": self.var_name,
            "Unit": self.unit}.update(self.year_dict)
        dict_df["2010"] = df_temp["2010"].sum(axis=0)  ## But why only 2010?
        df_temp_DEU = pd.DataFrame(dict_df, index = [0])

        neu = pd.concat([self.results, df_temp_DEU])

        for year in self.yearlist:
                df_year = df_temp[year]
                year_sum = df_year.sum()
                df_temp_DEU[year] = year_sum
        # Verbinde die bisherigen Results mit dem DF für Deutschland
        results_DEU = pd.concat([self.results, df_temp_DEU])
        self.results = results_DEU

    
    def change_unit(self, target_unit: str, operation: Optional[str], number: Optional[float] = 1):
        self.target_unit = target_unit
        self.operation = operation
        self.number = number
        df_temp = self.results
        print(df_temp)
        if type(operation)==type(lambda x: x):
            for year in self.yearlist:
                df_temp[year] = df_temp[year].apply(operation)            
        elif operation == "multiplication":
            for year in self.yearlist:
                df_temp[year] = df_temp[year].apply(lambda x: x*self.number)
        elif operation == "division":
            for year in self.yearlist:
                df_temp[year] = df_temp[year].apply(lambda x: x/self.number)

        df_temp["Unit"]=self.target_unit
        print(df_temp)
        self.results = df_temp

In [6]:
data.columns

Index(['Attribute', 'Commodity', 'Process', 'Period', 'Region', 'Vintage',
       'TimeSlice', 'UserConstraint', 'PV', 'Process_Description', 'MIMO',
       'Sector', 'Category1', 'Category2', 'Category3', 'Category4',
       'TechCapUnit', 'TechActUnit', 'Commodity_Description', 'Com_Sector',
       'Com_Type1', 'Com_Type2', 'Com_Type3', 'Com_Unit'],
      dtype='object')

In [34]:
## code to create the configuration file. 

columns = d.columns  # Read columns and create config file
config = {}
for i in range(len(d)):  # For all the values in dataframe, read non-empty vales and split.
    p = d.iloc[i]
    config[p[columns[0]]] = {}
    if "name" not in columns or "Name" not in columns:  # Create name if name of column name is not given
        config[p[columns[0]]]["Name"] = "|".join(str(p[columns[0]]).split("_")) 
    
    for col in columns[1:]:
        if (col=="name" or col=="Name") and config[p[columns[0]]].get("Name"):
            if pd.isnull(p[col]):
                config[p[columns[0]]]["Name"] = "|".join(str(p[columns[0]]).split("_"))
            else:
                config[p[columns[0]]]["Name"]=str(p[col])
            continue
        if not pd.isnull(p[col]):
            config[p[columns[0]]][col] = str(p[col]).split(",")

In [35]:
config

{'Emissions|CO2': {'Name': 'Emissions|CO2',
  'Class': ['single_var'],
  'Code': ['O-1-1'],
  'Unit': ['kt/yr'],
  'Attribute': ['VAR_FOut'],
  'Com_Type1': ['Environment'],
  'Com_Type2': ['CO2']},
 'Emissions|Industry|CO2': {'Name': 'Emissions|Industry|CO2',
  'Class': ['single_var'],
  'Code': ['O-1-2'],
  'Unit': ['kt/yr'],
  'Attribute': ['VAR_FOut'],
  'Commodity': ['INDCO2N', ' INDCO2P'],
  'Com_Type1': ['Environment'],
  'Com_Type2': ['CO2']},
 'Emissions|Transport|CO2': {'Name': 'Emissions|Transport|CO2',
  'Class': ['single_var'],
  'Code': ['O-1-3'],
  'Unit': ['kt/yr'],
  'Attribute': ['VAR_FOut'],
  'Commodity': ['TRACO2N', ' TRACO2P'],
  'Com_Type1': ['Environment'],
  'Com_Type2': ['CO2']},
 'Emissions|Residential|CO2': {'Name': 'Emissions|Residential|CO2',
  'Class': ['single_var'],
  'Code': ['O-1-4'],
  'Unit': ['kt/yr'],
  'Attribute': ['VAR_FOut'],
  'Commodity': ['RSDCO2N', ' RSDCO2P'],
  'Com_Type1': ['Environment'],
  'Com_Type2': ['CO2']},
 'Emissions|Commercial

In [36]:
single_var(
    template = False,
    var_name="Final Energy|Residential|Space Heating|Coal",
    unit ="PJ/yr",
    df_rawdata = filter_any(
        data,
        Region = region_list,
        Attribute = ["VAR_FIn"],
        Commodity = ["RSDCOA"],
        Category1 = ["Heating"],
        Sector = ["Residential"]
    )
).results

,Model,Scenario,Region,Variable,Unit,2010,2015,2025
0,TIMES PanEU v1.0,8GT_Bal,DER,Final Energy|Residential|Space Heating|Coal,PJ/yr,56.723174,10.95107,0.000377
0,TIMES PanEU v1.0,8GT_Bal,BW,Final Energy|Residential|Space Heating|Coal,PJ/yr,0.936771,0.00000,0.000000


In [37]:
%%time
output = {}
for k,v in config.items():
    output[k]={}
    var_name = v.pop("Name")  # The name is generated in the config file.
    unit = v.pop("Unit")[0]  # The unit also has to be scalar as it is not supported by single_var class for now.
    req_class = v.pop("Class")[0]  # This means that the class cannot be multiple
    code = v.pop("Code")  # Code to be used later on for arrangement.
    filtered_data = filter_any( data, **v )
    if req_class == "single_var":
        output[k]["raw"] = filtered_data
        output[k]['config'] = {"code": code, "req_class": req_class, "unit": unit, "var_name": var_name}
        output[k]["output"] = single_var(
            template = False,
            var_name=var_name,
            unit =unit,
            df_rawdata = filtered_data).results 
    else:
        print(f"{req_class} not implemented. Skipping {k}")

CPU times: user 32.1 s, sys: 1.69 s, total: 33.8 s
Wall time: 33.8 s


In [38]:
pd.Series(["v1", "v2", "v3"], index=[11,12,13], name="key")

11    v1
12    v2
13    v3
Name: key, dtype: object

In [39]:
pd.concat([pd.Series(["somevalue", "somevalue too"], name="somekey", index=None), output[k]["output"]], axis=0)

,0,Model,Scenario,Region,Variable,Unit,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,somevalue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,somevalue too,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,TIMES PanEU v1.0,8GT_Bal,DER,Final Energy|Residential|Other Demand|Electricity,PJ/yr,74.129997,73.179996,87.939995,91.529995,90.592205,87.095309,85.592244,72.377634,83.779937
0,NaN,TIMES PanEU v1.0,8GT_Bal,BW,Final Energy|Residential|Other Demand|Electricity,PJ/yr,12.020149,12.020149,14.867905,15.479059,15.315458,14.732501,14.466951,12.237327,14.172249


In [40]:
output.keys()

dict_keys(['Emissions|CO2', 'Emissions|Industry|CO2', 'Emissions|Transport|CO2', 'Emissions|Residential|CO2', 'Emissions|Commercial|CO2', 'Emissions|Electricity|CO2', 'Emissions|Supply|CO2', 'Emissions|Agriculture|CO2', 'Primary Energy|Natural Gas', 'Primary Energy|Oil', 'Primary Energy|Lignite', 'Primary Energy|Hard Coal', 'Primary Energy|Biomass', 'Primary Energy|Solar', 'Primary Energy|Wind', 'Primary Energy|Hydro', 'Primary Energy|Geothermal', 'Primary Energy|Waste', 'Primary Energy', 'Final Energy|Industry|Gases', 'Final Energy|Industry|Gases|Natural Gas', 'Final Energy|Industry|Gases|Synthetic Fossil', 'Final Energy|Industry|Gases|Biomass', 'Final Energy|Industry|Gases|Efuel', 'Final Energy|Industry|Liquids', 'Final Energy|Industry|Liquids|Biomass', 'Final Energy|Industry|Liquids|Petroleum', 'Final Energy|Industry|Liquids|Efuel', 'Final Energy|Industry|Solids', 'Final Energy|Industry|Solids|Biomass', 'Final Energy|Industry|Solids|Coal', 'Final Energy|Industry|Hydrogen', 'Final En

In [41]:
p = output["Emissions|CO2"]['output'].copy()
p

,Model,Scenario,Region,Variable,Unit,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,TIMES PanEU v1.0,8GT_Bal,DER,Emissions|CO2,kt/yr,726531.015372,711928.916936,593960.384004,471523.040883,344720.944866,225469.260248,101187.991042,-19651.049500,-16194.779359
0,TIMES PanEU v1.0,8GT_Bal,BW,Emissions|CO2,kt/yr,72160.999448,68490.797112,62167.581073,55156.349938,41420.544543,22157.599812,7689.262432,-44.207778,-359.118764


In [42]:
output["Emissions|CO2"]['config']

{'code': ['O-1-1'],
 'req_class': 'single_var',
 'unit': 'kt/yr',
 'var_name': 'Emissions|CO2'}

In [16]:
# The title of the excel should be var_name if exists otherwise it should be the key of the dictionary.
# code should be able to position the output in excel file. or a separate sheet.

In [46]:
writer = pd.ExcelWriter("output.xlsx", engine="xlsxwriter")
sheet = "some sheet"
offset = 2
for key in output.keys():
    for code in output[k]['config']['code']:
#         output[k]["output"].insert(0, "code", code*len(output[k]["output"]))
        output[key]["output"].to_excel(writer, sheet_name=sheet, startrow=offset, index=None)
        worksheet = writer.sheets[sheet]
        worksheet.write_string(0, 1, 'Your text here')
    offset += len(output[key])+1
    
writer.save()

/tmp/ipykernel_416/4164558987.py:12: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
